<a href="https://colab.research.google.com/github/ci5co150/CSCI191T/blob/main/Linear_Logistic_Regression_Programming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

By Francisco Gomez

Csci191T Fall 2021

In [ ]:
### Includes
%matplotlib inline
import csv
import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd

Import the csv data from github

In [ ]:
import pandas as pd
url = 'https://raw.githubusercontent.com/ci5co150/CSCI191T/main/iris.data.csv'
df1 = pd.read_csv(url)

data = df1.to_numpy().tolist()


Reformat the data from the csv file

In [ ]:
l = len(data)
# declare data X and data for Y (the results)
X = []
dataY = []
ones = [1]*len(data)
# Populate data X and dataY
for i in range(l):
  X.append(data[i][0:4])
  if (Y[i] == 'Iris-setosa'):
     dataY.append(0)
  elif (Y[i] == 'Iris-versicolor'):
    dataY.append(1)
  else:
    dataY.append(2) #Iris-virginica

dataX = [1]

for i in range(l):
  if (i == 0 ):
    dataX[0] = [ones[0]] + X[0]
  else:
    dataX.append([ones[i]] + X[i])



Setting up train and test data

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(dataX, dataY, test_size=0.33, random_state=45)

Create expected result data for 0 vs [1,2]

In [ ]:
# expected results for class 0 (Iris-setosa). Turn 0s to 1s (true) and the rest to 0's (false)
Y0 =  [y+1 if y==0 else y-y for y in dataY]
Y0_train = [y+1 if y==0 else y-y for y in Y_train]
Y0_test = [y+1 if y==0 else y-y for y in Y_test]

Create expected result data for 1 vs [1,2]

In [ ]:
# expected results for class 1 (Iris-versicolor). Keep 1s (true) and the rest to 0s (false)
Y1_train = [y if y==1 else y-y for y in Y_train]
Y1_test = [y if y==1 else y-y for y in Y_test]

Create expected result data fordata for 2 vs [0,1]

In [ ]:
# expected results for class 2 (Iris-virginica). Turn 2s to 1s (true) and the rest to 0s (false)
Y2_train = [y-1 if y==2 else y-y for y in Y_train]
Y2_test = [y-1 if y==2 else y-y for y in Y_test]

Create the tree models

In [ ]:
model0 = [0,0,0,0,0] # for class 0 (Iris-setosa)
model1 = [0,0,0,0,0] # for class 1 (Iris-versicolor)
model2 = [0,0,0,0,0] # for class 2 (Iris-virginica)

Simple Logistical Regression function

In [ ]:
def logReg(data,yHat,model):
  yprime =[]
  alpha = 0.01
  F = lambda W, X: sum([w*x for w,x in zip(W,X)])
  sigmoid = lambda y: 1/(1+math.exp(-y))
  classify = lambda y: 1 if y>0.5 else 0

  w = model

  X = [(d[0], d[1], d[2]) for d in data]

  Y = yHat
  for _ in range(500):
    yPredicted = [sigmoid(F(w, x)) for x in X]
    deltaX0 = sum( [((r-y)*x[0]) for r,y,x in zip(Y, yPredicted, X)] )
    deltaX1 = sum( [((r-y)*x[1]) for r,y,x in zip(Y, yPredicted, X)] )
    deltaX2 = sum( [((r-y)*x[2]) for r,y,x in zip(Y, yPredicted, X)] )
    
    w = [w[0]+alpha*deltaX0, w[1]+alpha*deltaX1, w[2]+alpha*deltaX2]

  return w

print("Model for Y0 | Iris-setosa ")
model0 = logReg(X_train,Y0_train,model0)
print(model0)

print("\nModel for Y1 | Iris-versicolor ")
model1 = logReg(X_train,Y1_train,model1)
print(model1)

print("\nModel for Y2 | Iris-virginica ")
model2 = logReg(X_train,Y2_train,model2)
print(model2)

Model for Y0 | Iris-setosa 
[1.334372957355326, -5.301260176457207, 8.654743925611163]

Model for Y1 | Iris-versicolor 
[7.604009731088582, 2.7970837299549673, -7.069031904619904]

Model for Y2 | Iris-virginica 
[-10.213570901203493, 6.91134514726345, -9.144031678531173]


Find the best model from the 3 for each test sample

In [ ]:
F = lambda W, X: sum([w*x for w,x in zip(W,X)])
sigmoid = lambda y: 1/(1+math.exp(-y))
classify = lambda y: 1 if y>0.5 else 0

X_test_prime = X_test
Y2_test_prime = Y2_test

yPredicted_with_model0 = ([round(sigmoid(F(model0, x)),3) for x in X_test],0)


yPredicted_with_model1 = ([round(sigmoid(F(model1, x)),3) for x in X_test],1)


yPredicted_with_model2 = ([round(sigmoid(F(model2, x)),3) for x in X_test],2)

#check for the "best" model to pick from the 3
predicted_test =[]
u = len(Y_test)
for i in range(u):
  if (yPredicted_with_model0[0][i] >= yPredicted_with_model1[0][i]  and 
      yPredicted_with_model0[0][i]  >= yPredicted_with_model2[0][i] ):
      predicted_test.append(yPredicted_with_model0[1])

  elif (yPredicted_with_model1[0][i]  >= yPredicted_with_model0[0][i]  and
      yPredicted_with_model1[0][i]  >= yPredicted_with_model2[0][i] ):
      predicted_test.append(yPredicted_with_model1[1])

  else:
      predicted_test.append(yPredicted_with_model2[1])

print("\n0 = Iris-setosa, 1 = Iris-versicolor,  2 = Iris-virginica" )
print("\nvaribles, Y Test , Y Predicted Test")
incorrect = 0
for i in range(u):
  if(Y_test[i] != predicted_test[i]):
    incorrect+=1
  print(X_test[i], Y_test[i], predicted_test[i] )
print("Errors: ", incorrect, "of ", u, "test samples.")


0 = Iris-setosa, 1 = Iris-versicolor,  2 = Iris-virginica

varibles, Y Test , Y Predicted Test
[1, 4.9, 3.0, 1.4, 0.2] 0 0
[1, 5.1, 3.8, 1.9, 0.4] 0 0
[1, 7.2, 3.0, 5.8, 1.6] 2 2
[1, 5.0, 3.6, 1.4, 0.2] 0 0
[1, 5.0, 3.2, 1.2, 0.2] 0 0
[1, 4.8, 3.0, 1.4, 0.3] 0 0
[1, 5.1, 3.4, 1.5, 0.2] 0 0
[1, 5.8, 2.8, 5.1, 2.4] 2 2
[1, 6.3, 2.7, 4.9, 1.8] 2 2
[1, 6.7, 3.1, 5.6, 2.4] 2 2
[1, 4.7, 3.2, 1.6, 0.2] 0 0
[1, 7.7, 3.0, 6.1, 2.3] 2 1
[1, 5.7, 2.5, 5.0, 2.0] 2 2
[1, 6.3, 2.9, 5.6, 1.8] 2 2
[1, 6.3, 2.8, 5.1, 1.5] 2 2
[1, 5.5, 3.5, 1.3, 0.2] 0 0
[1, 6.7, 3.0, 5.2, 2.3] 2 2
[1, 5.8, 2.7, 5.1, 1.9] 2 2
[1, 4.5, 2.3, 1.3, 0.3] 0 1
[1, 5.6, 3.0, 4.1, 1.3] 1 1
[1, 6.0, 3.4, 4.5, 1.6] 1 1
[1, 6.7, 3.1, 4.7, 1.5] 1 2
[1, 6.4, 3.2, 5.3, 2.3] 2 2
[1, 6.9, 3.1, 4.9, 1.5] 1 2
[1, 5.4, 3.9, 1.7, 0.4] 0 0
[1, 6.8, 3.0, 5.5, 2.1] 2 2
[1, 5.8, 2.6, 4.0, 1.2] 1 2
[1, 6.7, 3.1, 4.4, 1.4] 1 2
[1, 4.8, 3.4, 1.6, 0.2] 0 0
[1, 6.1, 3.0, 4.6, 1.4] 1 2
[1, 6.7, 3.0, 5.0, 1.7] 1 2
[1, 6.5, 3.0, 5.8, 2.2] 2 2
[1, 6.0,